# Olyset vs Untreated Pipeline

#### Pipeline Parameters

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_classif
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn import metrics
from statistics import mode
import openpyxl
import tqdm

import sys 
import joblib
import pickle
import itertools
import random

sys.path.append('H:/Documents/PhD/itns/olyset-vs-untreated/src/')
import extract, split, config

np.random.seed(0)
random.seed(0)

In [ ]:
# Segment Size and Overlap (in seconds)
segment_size = 6
segment_overlap = 5

# Trials split between test/train and validation set
test_trials = np.array([
    0,1, 4,5,6, 9,10, 13,14, 
    17,18,19, 23,24,25, 29,30,31, 35,36,37])
hyp_trials = np.array([
    2,3, 7,8, 11,12, 15,16,  
    20,21,22, 26,27,28, 32,33,34, 38,39])

# Paths
#results_path = config.PATH + 'tuned model/logistic-regression-mutual/' # Results stored
results_path = config.PATH + 'tests/13-run/' # Results stored
data_path = results_path + 'data/' # Any data 

#### Feature Extraction

##### Extract

In [ ]:
tracks, trackTargets, tracksTrialId, interpolated_flags = extract.load(config.FILE, config.PATH, config.IS_RESISTANT, config.DATA_PATH)

interpolated = []
for i in interpolated_flags:
    interpolated += i

with open(data_path + 'raw_tracks.npy', 'wb') as w:
    np.save(w, np.array(tracks, dtype=object))
with open(data_path + 'raw_trackTargets.npy', 'wb') as w:
    np.save(w, np.array(trackTargets, dtype=object))
with open(data_path + 'raw_tracksTrialId.npy', 'wb') as w:
    np.save(w, np.array(tracksTrialId, dtype=object))
with open(data_path + 'raw_interpolated.npy', 'wb') as w:
    np.save(w, np.array(interpolated, dtype=object))

In [ ]:
with open(data_path + 'raw_tracks.npy', 'wb') as w:
    np.save(w, np.array(tracks, dtype=object))
with open(data_path + 'raw_trackTargets.npy', 'wb') as w:
    np.save(w, np.array(trackTargets, dtype=object))
with open(data_path + 'raw_tracksTrialId.npy', 'wb') as w:
    np.save(w, np.array(tracksTrialId, dtype=object))
with open(data_path + 'raw_interpolated.npy', 'wb') as w:
    np.save(w, np.array(interpolated, dtype=object))

In [ ]:
tracks = np.load(data_path + 'raw_tracks.npy', allow_pickle=True)
trackTargets = np.load(data_path + 'raw_trackTargets.npy', allow_pickle=True)
tracksTrialId = np.load(data_path + 'raw_tracksTrialId.npy', allow_pickle=True)   

tracks = extract.generate_features(tracks, (0,1), 2)

with open(data_path + 'tracks_features.npy', 'wb') as w:
    np.save(w, tracks)

In [ ]:
tracks = np.load(data_path + 'tracks_features.npy', allow_pickle=True)

track_id = 0
while track_id < len(tracks):
    track_interpolated = np.array(interpolated[track_id]).astype(bool)
    tracks[track_id] = np.insert(tracks[track_id], len(tracks[track_id][0]), ~track_interpolated, axis=1)
    track_id += 1

with open(data_path + 'tracks_features_gaps_marked.npy', 'wb') as w:
    np.save(w, tracks)

In [ ]:
with open(data_path + 'tracks_features_gaps_marked.npy', 'wb') as w:
    np.save(w, tracks)

In [ ]:
tracks = np.load(data_path + 'tracks_features_gaps_marked.npy', allow_pickle=True)
trackTargets = np.load(data_path + 'raw_trackTargets.npy', allow_pickle=True)
tracksTrialId = np.load(data_path + 'raw_tracksTrialId.npy', allow_pickle=True)  

In [ ]:
indexes = []
for index, track in enumerate(tracks):
    # Negative times
    if np.any(track[:, 2] < 0):
        indexes.append(index)
    # Negative angular velocity
    elif np.any(track[:, 4] < 0):
        indexes.append(index)

tracks = np.delete(tracks, indexes, axis=0)
trackTargets = np.delete(trackTargets, indexes, axis=0)
tracksTrialId = np.delete(tracksTrialId, indexes, axis=0)

In [ ]:
tracks, trackTargets, tracksTrialId, trackGroup = split.split_tracks(
    tracks, trackTargets, tracksTrialId, segment_size, segment_overlap)

In [ ]:
def find_lowest_frame_for_trial(tracks, trials):
    unique_trials = np.unique(trials)
    frames = dict()
    for t in unique_trials:
        select = np.where(trials == t)[0]
        lowest = tracks[select[1]][0,16]

        for track in tracks[select]:
            try:
                small = min(track[:,16])
                if small < lowest:
                    lowest = small
            except:
                pass
        
        frames[f'{t}'] = lowest
    return frames

experiment_segment_time = []
frames = find_lowest_frame_for_trial(tracks, tracksTrialId)

for track_id, track in enumerate(tracks):
    first_track_time = frames[str(tracksTrialId[track_id])]
    earliest_time_in_exp = track[:, 16].min()
    adj_earliest_time_in_exp = (earliest_time_in_exp-first_track_time)/50

    experiment_segment_time.append(adj_earliest_time_in_exp)

In [ ]:
with open(data_path + 'tracks_split.npy', 'wb') as w:
    np.save(w, tracks)
with open(data_path + 'trackTargets_split.npy', 'wb') as w:
    np.save(w, trackTargets)
with open(data_path + 'trackGroup_split.npy', 'wb') as w:
    np.save(w, trackGroup)
with open(data_path + 'tracksTrialId_split.npy', 'wb') as w:
    np.save(w, tracksTrialId)

In [ ]:
tracks = np.load(data_path + 'tracks_split.npy', allow_pickle=True)
trackTargets = np.load(data_path + 'trackTargets_split.npy', allow_pickle=True)
trackGroup = np.load(data_path + 'trackGroup_split.npy', allow_pickle=True)  
tracksTrialId = np.load(data_path + 'tracksTrialId_split.npy', allow_pickle=True)  

In [ ]:
feature_columns = [
    'X Velocity',
    'Y Velocity',
    'X Acceleration', 
    'Y Acceleration',
    'Velocity',
    'Acceleration',
    'Jerk',
    'Angular Velocity',
    'Angular Acceleration',
    'Angle of Flight',
    'Centroid Distance Function',
    'Persistence Velocity',
    'Turning Velocity'
]   
indexes = [12,13,14,15,3,10,17,4,11,18,19,20,21]
feature_stats = [
    'mean','median','std', '1st quartile','3rd quartile','kurtosis', 'skewness',
    'number of local minima','number of local maxima','number of zero-crossings']     

track_statistics = dict()

for col in feature_columns:
    for stat in feature_stats:
        track_statistics[f'{col} ({stat})'] = []

for track in tracks:
    data = extract.track_stats(track, indexes=indexes, columns=feature_columns)
    for d in data:
        track_statistics[d].append(data[d])

df = pd.DataFrame(data=track_statistics)


In [ ]:
to_add = extract.add_other_features(tracks, (0,1))
df = pd.concat([df, to_add], axis=1)

df = df.join(pd.DataFrame({'TrialID': tracksTrialId}))

df_target = pd.DataFrame({
    'Target': trackTargets, 
    'TrialID': tracksTrialId, 
    'TrackGroup': trackGroup,
    'EarliestExpTime': experiment_segment_time
})

In [ ]:
df.to_pickle(data_path + 'df_raw.pkl')
df_target.to_pickle(data_path + 'df_target_raw.pkl')

In [ ]:
df

In [ ]:
df_target['EarliestExpTime'].sort_values()

##### Load Extract

In [ ]:
df = pd.read_pickle(data_path + 'df_raw.pkl')
df_target = pd.read_pickle(data_path + 'df_target_raw.pkl')
tracks = np.load(data_path + 'tracks_split.npy', allow_pickle=True)

###### Penalty Function

In [ ]:
def penalty_function(segment, n, m):
    penalty_score = 0
    k = 0

    for position in segment:
        if position == 0:
            penalty_score += n * (m ** k)
            k += 1
        else:
            k = max(0, k-1)

    return penalty_score/len(segment)


scores = []
for segment in tracks:
    mask = segment[:, -1]
    scores.append(penalty_function(mask, n=1, m=1.05))
scores = np.array(scores)

In [ ]:
max(scores)

In [ ]:
joblib.dump(scores, data_path + 'scores.dat')

In [ ]:
def create_threshold_list(start, end):
    current_value = start
    step_size = 1
    score_thresholds = []
    while current_value <= end:
        score_thresholds.append(int(current_value))
        if current_value < end/2:
            current_value += step_size
        else:
            current_value += min(step_size, 500)
        step_size = step_size * 1.1
    score_thresholds = np.array(score_thresholds)
    return score_thresholds


def run_score_threshold_mutual_info(df, df_target, scores):
    #score_thresholds = create_threshold_list(0, max(scores))
    score_thresholds = np.linspace(0, max(scores), 250)
    max_mutual_info_dict = {}
    unique_features = []
    for threshold in tqdm.tqdm(score_thresholds):
        mask = np.where(scores <= threshold)[0]
        df_temp = df.iloc[mask]
        df_target_temp = df_target.iloc[mask]

        indexes = df_temp[df_temp.isna().any(axis=1)].index
        df_temp = df_temp.drop(index=indexes)
        df_target_temp = df_target_temp.drop(index=indexes)

        df_temp = extract.remove_nans(df_temp)        

        df_temp = df_temp.drop(columns=['TrialID'])
        df_target_temp = df_target_temp['Target']

        mutual_info_values = mutual_info_classif(df_temp, df_target_temp)

        unique_features += df_temp.columns.values.tolist()

        for feature, mutual_info_value in zip(df_temp.columns, mutual_info_values):
            if feature not in max_mutual_info_dict or max_mutual_info_dict[feature]['mutual_info'] < mutual_info_value:
                max_mutual_info_dict[feature] = {
                    'mutual_info': mutual_info_value,
                    'score_threshold': threshold
                }
             
            elif max_mutual_info_dict[feature]['mutual_info'] == mutual_info_value and max_mutual_info_dict[feature]['score_threshold'] < threshold:
                max_mutual_info_dict[feature] = {
                    'mutual_info': mutual_info_value,
                    'score_threshold': threshold
                }

    unique_features = list(set(unique_features))
    corresponding_values = [max_mutual_info_dict[feature]['score_threshold'] for feature in unique_features]

    max_values = [max_mutual_info_dict[feature]['mutual_info'] for feature in unique_features]
    exp = np.exp(np.array(max_values)/(np.array(corresponding_values)+1))
    weights = exp / np.sum(exp)

    weighted_average_threshold = np.average(corresponding_values, weights=weights)

    return weighted_average_threshold

In [ ]:
score_threshold = run_score_threshold_mutual_info(
    df[df['TrialID'].isin(hyp_trials)],
    df_target[df_target['TrialID'].isin(hyp_trials)], 
    scores[df_target['TrialID'].isin(hyp_trials)])

In [ ]:
score_threshold

In [ ]:
788444627122805.5

In [ ]:
mask = np.where(scores <= score_threshold)[0]
df = df.iloc[mask]
df_target = df_target.iloc[mask]

indexes = df[df.isna().any(axis=1)].index
df = df.drop(index=indexes)
df_target = df_target.drop(index=indexes)

df = extract.remove_nans(df)  

In [ ]:
df.to_pickle(data_path + 'df_filtered.pkl')
df_target.to_pickle(data_path + 'df_target_filtered.pkl')

#### Split Train-Test/Validation sets

In [ ]:
df = pd.read_pickle(data_path + 'df_filtered.pkl')
df_target = pd.read_pickle(data_path + 'df_target_filtered.pkl')

In [ ]:
df_target['Banfora'] = 0
df_target['Kisumu'] = 0
df_target['Ngoussu'] = 0
df_target['VK7'] = 0

df_target.loc[df_target['TrialID'].isin([0,1,2,3, 17,18,19,20,21,22]), 'Banfora'] = 1
df_target.loc[df_target['TrialID'].isin([4,5,6,7,8, 23,24,25,26,27,28]),'Kisumu'] = 1
df_target.loc[df_target['TrialID'].isin([9,10,11,12, 29,30,31,32,33,34]),'Ngoussu'] = 1
df_target.loc[df_target['TrialID'].isin([13,14,15,16, 35,36,37,38,39]),'VK7'] = 1

df['Banfora'] = df_target['Banfora']
df['Kisumu'] = df_target['Kisumu']
df['Ngoussu'] = df_target['Ngoussu']
df['VK7'] = df_target['VK7']

In [ ]:
df_target['Resistant Status'] = 0

df_target.loc[df_target['TrialID'].isin([0,1,2,3, 17,18,19,20,21,22]), 'Resistant Status'] = 1
df_target.loc[df_target['TrialID'].isin([4,5,6,7,8, 23,24,25,26,27,28]),'Resistant Status'] = 0
df_target.loc[df_target['TrialID'].isin([9,10,11,12, 29,30,31,32,33,34]),'Resistant Status'] = 0
df_target.loc[df_target['TrialID'].isin([13,14,15,16, 35,36,37,38,39]),'Resistant Status'] = 1

df['Resistant Status'] = df_target['Resistant Status']

In [ ]:
df['Resistant Status'].value_counts()

In [ ]:
df_train = df[df['TrialID'].isin(test_trials)]
df_train_target = df_target[df_target['TrialID'].isin(test_trials)]

df_hyp = df[df['TrialID'].isin(hyp_trials)]
df_hyp_target = df_target[df_target['TrialID'].isin(hyp_trials)]

In [ ]:
df_hyp.to_pickle(data_path + 'df_hyp.pkl')
df_hyp_target.to_pickle(data_path + 'df_hyp_target.pkl')

#### Feature Selection

In [ ]:
df_hyp = df_hyp.drop(columns=['TrialID', 'Resistant Status'])

sus = df_hyp[df_hyp_target['Target'] == 0]
res = df_hyp[df_hyp_target['Target'] == 1]
_, p_val = mannwhitneyu(sus, res)
rej, p_vals_corrected, _, _ = multipletests(p_val, alpha=0.05, method='holm')

columns = df_hyp.columns[rej]

df_hyp = df_hyp.reset_index(drop=True)
corr_matrix = df_hyp.corr(method='spearman').abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.85)]

cols = np.setdiff1d(columns, to_drop)

In [ ]:
features = []
for feat in cols:
    if ('TrialID' not in feat):
        features.append(feat)

In [ ]:
features = []
for feat in cols:
    if ('TrialID' not in feat):
        features.append(feat)

features += ['Resistant Status'] #['Kisumu', 'Banfora', 'Ngoussu', 'VK7']

In [ ]:
file = open(results_path + 'features.txt', 'w+')
for feat in features:
    file.write(feat+'\n')

file.close()

In [ ]:
len(features)

In [ ]:
features = open(results_path + 'features.txt', 'r+').read().split('\n')
features.remove('')

In [ ]:
for i in sorted(features):
    print(i)

## Model

#### Create Folds

In [ ]:
def create_folds():
    final_folds = []
    for v1 in list(itertools.product([0,1], [4,5,6], [9,10], [13,14])):
        for v2 in list(itertools.product([17,18,19], [23,24,25], [29,30,31], [35,36,37])):
            train_trials = list(v1) + list(v2)
            final_folds.append(train_trials)
    return final_folds

In [ ]:
def create_folds():
    final_folds = []
    for v1 in itertools.product([0, 1], [4, 5, 6], [9, 10], [13, 14]):  # UT
        for v2 in itertools.product(                                    # OL
                itertools.combinations([17, 18, 19], 2), 
                itertools.combinations([23, 24, 25], 2), 
                itertools.combinations([29, 30, 31], 2), 
                itertools.combinations([35, 36, 37], 2)
        ):
            train_trials = list(v1) + [item for sublist in v2 for item in sublist]
            final_folds.append(train_trials)
    
    return final_folds

In [ ]:
folds = create_folds()
folds = random.sample(folds, 30)

In [ ]:
file = open(results_path+'folds.txt', 'w+')
for f in folds:
    file.write(str(f) +'\n')
file.close()

In [ ]:
file = open(results_path+'folds.txt', 'r+').read()
file_split = file.split('\n')
file_split.remove('')
folds = [eval(i) for i in file_split]

In [ ]:
len(folds)

#### Model Building

In [ ]:
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return TP, FP, TN, FN


def get_track_prediction(y_true, scores, preds, groups):
    unique_groups = groups.unique()
    track_preds = []
    track_true = []
    avg_scores = []
    for val in unique_groups:
        indexes = np.where(groups == val)[0]
        track_true.append(mode(y_true.values[indexes]))
        avg_scores.append(np.mean(scores[indexes]))
        if np.mean(preds[indexes]) >= 0.5: 
            track_preds.append(1)
        else:
            track_preds.append(0)

    return track_true, track_preds, avg_scores


def produce_report(y_test, y_pred, scores):
    precision_ir, recall_ir, _ = metrics.precision_recall_curve(y_test, scores, pos_label=1)
    precision_is, recall_is, _ = metrics.precision_recall_curve(y_test, scores, pos_label=0)
    
    TP, FP, TN, FN = perf_measure(y_test, y_pred)
    fpr, tpr, _ = metrics.roc_curve(y_test, scores)
    data = {
        'accuracy': metrics.accuracy_score(y_test,y_pred),
        'balanced accuracy': metrics.balanced_accuracy_score(y_test,y_pred),
        'roc auc': metrics.roc_auc_score(y_test, scores),

        'f1 score (OL)': metrics.f1_score(y_test,y_pred, labels=np.unique(y_test), pos_label=1),
        'precision (OL)': metrics.precision_score(y_test, y_pred, labels=np.unique(y_test), pos_label=1),
        'recall (OL)': metrics.recall_score(y_test, y_pred, labels=np.unique(y_test), pos_label=1),
        'pr auc (OL)': metrics.auc(recall_ir, precision_ir),

        'f1 score (UT)': metrics.f1_score(y_test, y_pred, labels=np.unique(y_test), pos_label=0),
        'precision (UT)': metrics.precision_score(y_test, y_pred, labels=np.unique(y_test), pos_label=0),
        'recall (UT)': metrics.recall_score(y_test, y_pred, labels=np.unique(y_test), pos_label=0),
        'pr auc (UT)': metrics.auc(recall_is, precision_is),

        'cohen kappa': metrics.cohen_kappa_score(np.array(y_test), np.array(y_pred)),
        'matthews correlation coefficient': metrics.matthews_corrcoef(np.array(y_test), np.array(y_pred)),
        'log loss': metrics.log_loss(np.array(y_test), np.array(scores)),
        'brier score': metrics.brier_score_loss(y_test, scores),
        'tp': TP, 'tn': TN, 'fp': FP, 'fn': FN,
        'tpr': tpr, 'fpr': fpr
    }
    return data

In [ ]:
def constant_model(x_train, y_train, x_test, y_test):
    model = DummyRegressor(strategy='constant', constant=0)
    model.fit(x_train, y_train['Target'])
    y_pred = model.predict(x_test)
    track_true, track_preds, scores = get_track_prediction(
        y_test['Target'], y_pred, y_pred, y_test['TrackGroup'])
    return produce_report(track_true, track_preds, scores), {
        'segment-preds': y_pred, 
        'segment-target': y_test['Target'],
        'segment-trial-ids': y_test['TrialID'],
        'segment-track-group': y_test['TrackGroup'],
        'segment-scores': y_pred
    }  


def xgboost_model(x_train, y_train, x_test, y_test, params, thresholds):
    model = XGBClassifier(
        **params
    )
    model.fit(x_train, y_train['Target'])
    scores = model.predict_proba(x_test)[:,1]

    if type(thresholds) == list:
        results = []
        for threshold in thresholds:
            y_pred = (scores > threshold).astype(int)
            track_true, track_preds, avg_scores = get_track_prediction(
                y_test['Target'], scores, y_pred, y_test['TrackGroup'])
            results.append((produce_report(track_true, track_preds, avg_scores), {
                'segment-preds': y_pred, 
                'segment-target': y_test['Target'],
                'segment-trial-ids': y_test['TrialID'],
                'segment-track-group': y_test['TrackGroup'],
                'segment-scores': scores
            }, model))
        return results
    elif type(thresholds) == float:
        y_pred = (scores > thresholds).astype(int)
        track_true, track_preds, avg_scores = get_track_prediction(
            y_test['Target'], scores, y_pred, y_test['TrackGroup'])
        return (produce_report(track_true, track_preds, avg_scores), {
            'segment-preds': y_pred, 
            'segment-target': y_test['Target'],
            'segment-trial-ids': y_test['TrialID'],
            'segment-track-group': y_test['TrackGroup'],
            'segment-scores': scores
        }, model)
    raise ValueError('threshold wrong')


In [ ]:
# Function to undersample mask
def undersample_mask(targets, mask, num_samples):
    track_groups = targets[mask]['TrackGroup'].unique()
    if len(track_groups) > num_samples:
        sampled_track_groups = np.random.choice(track_groups, num_samples, replace=False)
        track_groups_mask = targets['TrackGroup'].isin(sampled_track_groups)
        return track_groups_mask & mask
    else:
        return mask

In [ ]:
np.mean((5, 2))

In [ ]:
results = dict(
    dummy_results_train = [],
    dummy_results_test = [],
    xgboost_train = [],
    xgboost_test = [],
)

segment_scores_constant_train = []
segment_scores_constant_test = []

segment_scores_xgboost_train = []
segment_scores_xgboost_test = []

fold_threshold_results = []


for index, fold in enumerate(folds):
    print(f' --- FOLD {index} ---')
    train_trials = fold
    mask = df_train_target['TrialID'].isin(train_trials)

    train = df_train[mask]
    train_targets = df_train_target[mask]
    
    test = df_train[~mask]
    test_targets = df_train_target[~mask]

    # For OL train trials - before 30mins
    # For UT train trials - randomly selected
    ol_banfora_train_mask = train_targets['TrialID'].isin([17,18,19,20,21,22])
    ol_kisumu_train_mask = train_targets['TrialID'].isin([23,24,25,26,27,28]) & (train_targets['EarliestExpTime'] <= 30*60)
    ol_ngoussu_train_mask = train_targets['TrialID'].isin([29,30,31,32,33,34]) & (train_targets['EarliestExpTime'] <= 30*60)
    ol_vk7_train_mask = train_targets['TrialID'].isin([35,36,37,38,39])

    ut_banfora_train_mask = train_targets['TrialID'].isin([0,1,2,3])
    ut_kisumu_train_mask = train_targets['TrialID'].isin([4,5,6,7,8,])
    ut_ngoussu_train_mask = train_targets['TrialID'].isin([9,10,11,12])
    ut_vk7_train_mask = train_targets['TrialID'].isin([13,14,15,16])

    num_samples_kisumu =  len(train_targets[ol_kisumu_train_mask]['TrackGroup'].unique())
    num_samples_ngoussu = len(train_targets[ol_ngoussu_train_mask]['TrackGroup'].unique())
    num_samples = np.mean((num_samples_kisumu, num_samples_ngoussu)).astype(int)

    # Undersample the masks
    ol_banfora_train_mask_undersampled = undersample_mask(train_targets, ol_banfora_train_mask, num_samples)
    ol_vk7_train_mask_undersampled = undersample_mask(train_targets, ol_vk7_train_mask, num_samples)

    ut_banfora_train_mask_undersampled = undersample_mask(train_targets, ut_banfora_train_mask, num_samples)
    ut_kisumu_train_mask_undersampled = undersample_mask(train_targets, ut_kisumu_train_mask, num_samples)
    ut_ngoussu_train_mask_undersampled = undersample_mask(train_targets, ut_ngoussu_train_mask, num_samples)
    ut_vk7_train_mask_undersampled = undersample_mask(train_targets, ut_vk7_train_mask, num_samples)

    train_mask = ol_kisumu_train_mask | ol_ngoussu_train_mask | ol_banfora_train_mask_undersampled | ol_vk7_train_mask_undersampled | ut_banfora_train_mask_undersampled | ut_kisumu_train_mask_undersampled | ut_ngoussu_train_mask_undersampled | ut_vk7_train_mask_undersampled
    train = train[train_mask]
    train_targets = train_targets[train_mask]

    # For OL test trials - before 30 mins
    # For UT train trials - whole trial
    ol_is_test_mask = test_targets['TrialID'].isin([23,24,25,26,27,28,29,30,31,32,33,34]) & (test_targets['EarliestExpTime'] <= 30*60)
    ol_ir_test_mask = test_targets['TrialID'].isin([17,18,19,20,21,22,35,36,37,38,39])
    ut_is_test_mask = test_targets['TrialID'].isin([4,5,6,7,8,9,10,11,12])
    ut_ir_test_mask = test_targets['TrialID'].isin([0,1,2,3,13,14,15,16])

    test_mask = ol_is_test_mask | ol_ir_test_mask | ut_is_test_mask | ut_ir_test_mask
    test = test[test_mask]
    test_targets = test_targets[test_mask]

    scaler = StandardScaler()
    train = pd.DataFrame(scaler.fit_transform(train[features]), columns=features, index=train.index)
    test = pd.DataFrame(scaler.transform(test[features]), columns=features, index=test.index)

    #sm = SMOTE(
    #    random_state=0
    #)
    #train_os, train_targets_os = sm.fit_resample(train, train_targets.drop(columns=['TrialID','TrackGroup']))
    train_os = train
    train_targets_os = train_targets.drop(columns=['TrialID', 'TrackGroup'])

    constant_scores, constant_segment_scores = constant_model(
        x_train=train_os, 
        y_train=train_targets_os, 
        x_test=train, 
        y_test=train_targets)
    results['dummy_results_train'].append(constant_scores)
    segment_scores_constant_train.append(constant_segment_scores)

    constant_scores, constant_segment_scores = constant_model(
        x_train=train_os, 
        y_train=train_targets_os, 
        x_test=test, 
        y_test=test_targets)

    results['dummy_results_test'].append(constant_scores)
    segment_scores_constant_test.append(constant_segment_scores)
    

    negative_examples = train_targets_os[train_targets_os['Target'] == 0]
    positive_examples = train_targets_os[train_targets_os['Target'] == 1]
    scale_pos_weight = len(negative_examples) / len(positive_examples)


    thresholds = np.arange(0.01, 1, 0.01).tolist()
    threshold_results = xgboost_model(
        x_train=train_os, 
        y_train=train_targets_os, 
        x_test=train, 
        y_test=train_targets,
        params=dict(
            random_state=0,
            learning_rate=0.1,
            n_estimators=200,
            max_depth=7,
            subsample=0.7,
            colsample_bytree=0.9,
            reg_alpha=0.01, 
            reg_lambda=0.1,
            min_child_weight=5,
            scale_pos_weight=scale_pos_weight   
        ),
        thresholds=thresholds)

    best_index = np.argmax([i[0]['matthews correlation coefficient'] for i in threshold_results])
    results['xgboost_train'].append(threshold_results[best_index][0])
    segment_scores_xgboost_train.append(threshold_results[best_index][1])
    best_threshold = thresholds[best_index]
    print(best_threshold)

    fold_threshold_results.append(threshold_results)

    xg_scores, segment_scores, model = xgboost_model(
        x_train=train_os, 
        y_train=train_targets_os, 
        x_test=test, 
        y_test=test_targets,
        params=dict(
            random_state=0,
            learning_rate=0.1,
            n_estimators=200,
            max_depth=7,
            subsample=0.7,
            colsample_bytree=0.9,
            reg_alpha=0.01, 
            reg_lambda=0.1,
            min_child_weight=5,
            scale_pos_weight=scale_pos_weight 
        ),
        thresholds=best_threshold)
    results['xgboost_test'].append(xg_scores)
    segment_scores_xgboost_test.append(segment_scores)
    
    joblib.dump(dict(
        model=model,
        df_train=df_train,
        df_train_target=df_train_target,
        features=features,
        test=test,
        test_targets=test_targets,
        mask=mask,
        train_os=train_os,
    ), data_path+f'shap/xgboost_shap_dump_{index}.dat')

In [ ]:

def produce_report(y_test, y_pred, scores):
    precision_ir, recall_ir, _ = metrics.precision_recall_curve(y_test, scores, pos_label=1)
    precision_is, recall_is, _ = metrics.precision_recall_curve(y_test, scores, pos_label=0)
    
    TP, FP, TN, FN = perf_measure(y_test, y_pred)
    fpr, tpr, _ = metrics.roc_curve(y_test, scores)
    data = {
        'accuracy': metrics.accuracy_score(y_test,y_pred),
        'balanced accuracy': metrics.balanced_accuracy_score(y_test,y_pred),
        'roc auc': metrics.roc_auc_score(y_test, scores),

        'f1 score (OL)': metrics.f1_score(y_test,y_pred, labels=np.unique(y_test), pos_label=1),
        'precision (OL)': metrics.precision_score(y_test, y_pred, labels=np.unique(y_test), pos_label=1),
        'recall (OL)': metrics.recall_score(y_test, y_pred, labels=np.unique(y_test), pos_label=1),
        'pr auc (OL)': metrics.auc(recall_ir, precision_ir),

        'f1 score (UT)': metrics.f1_score(y_test, y_pred, labels=np.unique(y_test), pos_label=0),
        'precision (UT)': metrics.precision_score(y_test, y_pred, labels=np.unique(y_test), pos_label=0),
        'recall (UT)': metrics.recall_score(y_test, y_pred, labels=np.unique(y_test), pos_label=0),
        'pr auc (UT)': metrics.auc(recall_is, precision_is),

        'cohen kappa': metrics.cohen_kappa_score(np.array(y_test), np.array(y_pred)),
        'matthews correlation coefficient': metrics.matthews_corrcoef(np.array(y_test), np.array(y_pred)),
        'log loss': metrics.log_loss(np.array(y_test), np.array(scores)),
        'brier score': metrics.brier_score_loss(y_test, scores),
        'tp': TP, 'tn': TN, 'fp': FP, 'fn': FN,
        'tpr': tpr, 'fpr': fpr
    }
    return data

strain_results = dict(
    banfora_train = [],
    banfora_test = [],
    kisumu_train = [],
    kisumu_test = [],
    ngoussu_train = [],
    ngoussu_test = [],
    vk7_train = [],
    vk7_test = []
)
for data in segment_scores_xgboost_test:
    # Banfora
    mask = data['segment-trial-ids'].isin([0,1,2,3, 17,18,19,20,21,22])
    track_true, track_preds, avg_scores = get_track_prediction(
            data['segment-target'][mask], 
            data['segment-preds'][mask], 
            data['segment-scores'][mask], 
            data['segment-track-group'][mask])
    strain_results['banfora_test'].append(produce_report(track_true, track_preds, avg_scores))

    # Kisumu
    mask = data['segment-trial-ids'].isin([4,5,6,7,8, 23,24,25,26,27,28])
    track_true, track_preds, avg_scores = get_track_prediction(
            data['segment-target'][mask], 
            data['segment-preds'][mask], 
            data['segment-scores'][mask], 
            data['segment-track-group'][mask])
    strain_results['kisumu_test'].append(produce_report(track_true, track_preds, avg_scores))

    # Ngoussu
    mask = data['segment-trial-ids'].isin([9,10,11,12, 29,30,31,32,33,34])
    track_true, track_preds, avg_scores = get_track_prediction(
            data['segment-target'][mask], 
            data['segment-preds'][mask], 
            data['segment-scores'][mask], 
            data['segment-track-group'][mask])
    strain_results['ngoussu_test'].append(produce_report(track_true, track_preds, avg_scores))

    # VK7
    mask = data['segment-trial-ids'].isin([13,14,15,16, 35,36,37,38,39])
    track_true, track_preds, avg_scores = get_track_prediction(
            data['segment-target'][mask], 
            data['segment-preds'][mask], 
            data['segment-scores'][mask], 
            data['segment-track-group'][mask])
    strain_results['vk7_test'].append(produce_report(track_true, track_preds, avg_scores))

for data in segment_scores_xgboost_train:
    # Banfora
    mask = data['segment-trial-ids'].isin([0,1,2,3, 17,18,19,20,21,22])
    track_true, track_preds, avg_scores = get_track_prediction(
            data['segment-target'][mask], 
            data['segment-preds'][mask], 
            data['segment-scores'][mask], 
            data['segment-track-group'][mask])
    strain_results['banfora_train'].append(produce_report(track_true, track_preds, avg_scores))

    # Kisumu
    mask = data['segment-trial-ids'].isin([4,5,6,7,8, 23,24,25,26,27,28])
    track_true, track_preds, avg_scores = get_track_prediction(
            data['segment-target'][mask], 
            data['segment-preds'][mask], 
            data['segment-scores'][mask], 
            data['segment-track-group'][mask])
    strain_results['kisumu_train'].append(produce_report(track_true, track_preds, avg_scores))

    # Ngoussu
    mask = data['segment-trial-ids'].isin([9,10,11,12, 29,30,31,32,33,34])
    track_true, track_preds, avg_scores = get_track_prediction(
            data['segment-target'][mask], 
            data['segment-preds'][mask], 
            data['segment-scores'][mask], 
            data['segment-track-group'][mask])
    strain_results['ngoussu_train'].append(produce_report(track_true, track_preds, avg_scores))

    # VK7
    mask = data['segment-trial-ids'].isin([13,14,15,16, 35,36,37,38,39])
    track_true, track_preds, avg_scores = get_track_prediction(
            data['segment-target'][mask], 
            data['segment-preds'][mask], 
            data['segment-scores'][mask], 
            data['segment-track-group'][mask])
    strain_results['vk7_train'].append(produce_report(track_true, track_preds, avg_scores))


In [ ]:
ms = [
    'balanced accuracy', 'f1 score (OL)',
    'f1 score (UT)',  'cohen kappa', 'matthews correlation coefficient',
    'log loss', 'brier score']

for m in ms:
    plt.figure()
    for res in fold_threshold_results:
        plt.plot(np.arange(0.01, 1, 0.01), [i[0][m] for i in res], alpha=0.8)
    plt.xlabel('Decision score threshold')
    plt.ylabel(m)
    plt.title(f'Change in {m} for varying decision score threshold')
    plt.show()

In [ ]:
with open(results_path+'results.pkl', 'wb') as f:
    pickle.dump(results, f)

with open(results_path+'scores.pkl', 'wb') as f:
    pickle.dump(segment_scores_xgboost_test, f)

In [ ]:
file = open(results_path+'folds.txt', 'w+')
for fold in folds:
    file.write(str(fold) +'\n')
file.close()

In [ ]:
with open(results_path+'results.pkl', 'rb') as f:
    results = pickle.load(f)

#### Model Performance

In [ ]:
wb = openpyxl.Workbook()
sheet = wb.create_sheet()

row = 2
metrics_list = list(results['xgboost_train'][0].keys())
for i, column in enumerate(['model'] + metrics_list):
    sheet.cell(row=1, column=i+1).value = column

for model in ['dummy_results', 'xgboost']:
    for model_type in ['train', 'test']:
        try:
            sheet.cell(row=row, column=1).value = f'{model.upper()} {model_type.upper()}'
            for j, metric in enumerate(results[model+'_'+model_type][0].keys()):
                if metric in ['tp', 'tn', 'fp', 'fn']:
                    scores = 0
                    for fold in range(len(results[model+'_'+model_type])):
                        scores += results[model+'_'+model_type][fold][metric]
                    
                    sheet.cell(row=row, column=j+2).value = scores 
                elif metric in ['tpr', 'fpr']:
                    pass
                else:
                    scores = []
                    for fold in range(len(results[model+'_'+model_type])):
                        scores.append(results[model+'_'+model_type][fold][metric])
                    
                    sheet.cell(row=row, column=j+2).value = f'{round(np.mean(scores), 3)} ({round(min(scores), 3)} - {round(max(scores), 3)})'
            row += 1
        except:
            pass

wb.save(results_path + 'scores.xlsx')

In [ ]:
wb = openpyxl.Workbook()
sheet = wb.create_sheet()

row = 2
metrics_list = list(strain_results['banfora_train'][0].keys())
for i, column in enumerate(['model'] + metrics_list):
    sheet.cell(row=1, column=i+1).value = column

for model in ['banfora', 'kisumu', 'ngoussu', 'vk7']:
    for model_type in ['train', 'test']:
        try:
            sheet.cell(row=row, column=1).value = f'{model.upper()} {model_type.upper()}'
            for j, metric in enumerate(strain_results[model+'_'+model_type][0].keys()):
                if metric in ['tp', 'tn', 'fp', 'fn']:
                    scores = 0
                    for fold in range(len(strain_results[model+'_'+model_type])):
                        scores += strain_results[model+'_'+model_type][fold][metric]
                    
                    sheet.cell(row=row, column=j+2).value = scores 
                elif metric in ['tpr', 'fpr']:
                    pass
                else:
                    scores = []
                    for fold in range(len(strain_results[model+'_'+model_type])):
                        scores.append(strain_results[model+'_'+model_type][fold][metric])
                    
                    sheet.cell(row=row, column=j+2).value = f'{round(np.mean(scores), 3)} ({round(min(scores), 3)} - {round(max(scores), 3)})'
            row += 1
        except:
            pass

wb.save(results_path + 'scores-strain.xlsx')

#### Graphs

In [ ]:
'''CONFUSION MATRIX'''

def plot_confusion_matrix(tp, tn, fp, fn, classifier):
    confusion_matrix = np.array([[tn, fp], [fn, tp]])

    fig, ax = plt.subplots(dpi=300)

    im = ax.imshow(confusion_matrix, cmap='Oranges')

    ax.set_xlabel('Predicted Class')
    ax.set_ylabel('True Class')

    ax.set_xticks([0, 1])
    ax.set_yticks([0, 1])
    ax.set_xticklabels(['IS', 'IR'])
    ax.set_yticklabels(['IS', 'IR'])

    for i in range(2):
        for j in range(2):
            ax.text(j, i, confusion_matrix[i, j], ha='center', va='center', fontsize=14)

    plt.show()

def plot_confusion_matrix_percent(tp, tn, fp, fn, classifier):
    confusion_matrix = np.array([[tn, fp], [fn, tp]])

    fig, ax = plt.subplots(dpi=300)

    im = ax.imshow(confusion_matrix, cmap='Oranges')

    ax.set_xlabel('Predicted Class')
    ax.set_ylabel('True Class')

    ax.set_xticks([0, 1])
    ax.set_yticks([0, 1])
    ax.set_xticklabels(['UT', 'OL'])
    ax.set_yticklabels(['UT', 'OL'])

    total_samples = np.sum(confusion_matrix)
    for i in range(2):
        for j in range(2):
            percentage = confusion_matrix[i, j] / total_samples * 100
            ax.text(j, i, f'{percentage:.2f}%', ha='center', va='center', color='black', fontsize=14)
    plt.show()


for key in ['dummy_results_train', 'dummy_results_test','xgboost_train', 'xgboost_test']:
    print(key)
    plot_confusion_matrix_percent(
        sum([_['tp'] for _ in results[key]]), 
        sum([_['tn'] for _ in results[key]]), 
        sum([_['fp'] for _ in results[key]]), 
        sum([_['fn'] for _ in results[key]]),
        classifier=key
    )

In [ ]:
'''ROC CURVES'''

def plot_roc_curve(tpr, fpr, classifier):
    fig, ax = plt.subplots(dpi=300)

    for i in range(len(tpr)):
        ax.plot(fpr[i], tpr[i], alpha=0.5)

    ax.plot([0, 1], [0, 1], linestyle='--')

    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')

    plt.show()


for key in ['dummy_results_train', 'dummy_results_test','xgboost_train', 'xgboost_test']:
    print(key)
    plot_roc_curve(
        [_['tpr'] for _ in results[key]],  
        [_['fpr'] for _ in results[key]], 
        classifier=key
    )

In [ ]:
'''PR CURVES'''

def plot_pr_curve(precision, recall, classifier):
    fig, ax = plt.subplots()

    base = np.linspace(0, 1, 101)
    avg_precision = 1

    for i in range(len(precision)):
        ax.plot(recall[i], precision[i])
        new_avg_precision = sum(precision[i]) / len(precision[i])
        if new_avg_precision < avg_precision:
            avg_precision = new_avg_precision

    ax.plot(base, [avg_precision]*len(base), linestyle='--')

    ax.set_xlabel('Recall')
    ax.set_ylabel('Precision')
    ax.set_title(f'PR Curves - {classifier}')

    plt.show()


for key in ['dummy_results_train', 'dummy_results_test','xgboost_train','xgboost_test']:
    plot_pr_curve(
        [results[key][i]['precision (ir) (curve)'] for i in range(len(results[key]))],
        [results[key][i]['recall (ir) (curve)'] for i in range(len(results[key]))], 
        classifier=key+' (OL)'
    )
    plot_pr_curve(
        [results[key][i]['precision (is) (curve)'] for i in range(len(results[key]))],
        [results[key][i]['recall (is) (curve)'] for i in range(len(results[key]))], 
        classifier=key+' (UT)'
    )

In [ ]:
'''EXCEL FILE OF ALL FOLD SCORES'''

wb = openpyxl.Workbook()
for key in ['dummy_results_train', 'dummy_results_test','xgboost_train','xgboost_test']:
    sheet = wb.create_sheet(key.upper())
    columns = ['fold', 'test trials', 'train trials', 'accuracy', 'balanced accuracy',
    'f1 score (OL)', 'roc auc', 'precision (OL)', 'recall (OL)', 'pr auc (OL)',
    'f1 score (UT)', 'precision (UT)', 'recall (UT)', 'pr auc (UT)',
    'cohen kappa', 'matthews correlation coefficient', 'log loss', 'tp', 'tn', 'fp', 'fn']
    for i, column in enumerate(columns):
        sheet.cell(row=1, column=i+1).value = column

        for row in range(len(results[key])):
            if column not in ['fold', 'test trials', 'train trials']:
                sheet.cell(row=row+2, column=i+1).value = results[key][row][column]
        
            elif column == 'fold':
                sheet.cell(row=row+2, column=i+1).value = row

            elif column == 'test trials':
                train = folds[row]
                all_ids = df_train_target['TrialID'].unique()
                sheet.cell(row=row+2, column=i+1).value = str([x for x in all_ids if x not in train])

            elif column == 'train trials':
                sheet.cell(row=row+2, column=i+1).value = str(folds[row])

wb.save(results_path + "all-folds.xlsx")